In [6]:
import tensorflow as tf
from datasets import load_from_disk
from transformers import TFAutoModelForMaskedLM, AutoTokenizer, DistilBertTokenizerFast

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = DistilBertTokenizerFast.from_pretrained('test-tokenizer-cleaned-30000')
# tokenized_datasets = load_from_disk('distilbert-model/tokenized-dataset')
lm_datasets = load_from_disk('distilbert-model/lm_dataset-30000')
dataset = load_from_disk("cleaned-data-mc4-v2")


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [4]:
# from tokenizers.trainers import BpeTrainer, WordPieceTrainer
# trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [6]:
# from tokenizers.pre_tokenizers import Whitespace
# tokenizer.pre_tokenizer = Whitespace()

In [2]:
# train_size = 10000
# test_size = int(0.1 * train_size)
#
# downsampled_dataset = dataset["train"].train_test_split(
#     train_size=train_size, test_size=test_size, seed=42
# )
# downsampled_dataset

Loading cached split indices for dataset at C:\Users\Davit6174\PycharmProjects\huggingface\web-crawl-mc4\train\cache-1808b37876d061fd.arrow and C:\Users\Davit6174\PycharmProjects\huggingface\web-crawl-mc4\train\cache-2d9403e9c6be26c7.arrow


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

In [34]:
# from tokenizers import trainers
# def batch_iterator(batch_size=100):
#     for i in range(0, len(downsampled_dataset), batch_size):
#         yield dataset[i : i + batch_size]["text"]
#
# special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
# trainer = trainers.WordPieceTrainer(min_frequency=3, special_tokens=special_tokens)
# tokenizer.train_new_from_iterator(downsampled_dataset, vocab_size=30000, min_frequency=3, special_tokens=special_tokens)

TypeError: argument 'files': 'DatasetDict' object cannot be converted to 'Sequence'

In [33]:
# from transformers import DistilBertTokenizerFast
#
# # Pass the vocabulary and merges to DistilBertTokenizerFast
# distilbert_tokenizer = DistilBertTokenizerFast(tokenizer_object=tokenizer)


AttributeError: 'NoneType' object has no attribute '__getstate__'

In [2]:
from datasets import concatenate_datasets


def get_training_corpus():
    data = concatenate_datasets([dataset["train"], dataset["test"]])
    for start_idx in range(0, len(dataset), 1000):
        samples = data[start_idx : start_idx + 1000]
        yield samples["text"]

training_corpus = get_training_corpus()

In [3]:
tokenizer.is_fast

True

In [4]:
new_tokenizer = tokenizer.train_new_from_iterator(training_corpus, 30000)
new_tokenizer.save_pretrained("test-tokenizer-cleaned-30000")

('test-tokenizer-cleaned-30000\\tokenizer_config.json',
 'test-tokenizer-cleaned-30000\\special_tokens_map.json',
 'test-tokenizer-cleaned-30000\\vocab.txt',
 'test-tokenizer-cleaned-30000\\added_tokens.json',
 'test-tokenizer-cleaned-30000\\tokenizer.json')

In [10]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('test-tokenizer-cleaned-30000')

In [6]:
# tokenizer = test_tokenizer

In [4]:
tokenizer.is_fast

True

In [13]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets.save_to_disk('distilbert-model/tokenized-dataset-30000')
tokenized_datasets

Map:   0%|          | 0/1836440 [00:00<?, ? examples/s]

Map:   0%|          | 0/459111 [00:00<?, ? examples/s]

Map:   0%|          | 0/2279 [00:00<?, ? examples/s]

Saving the dataset (0/34 shards):   0%|          | 0/1836440 [00:00<?, ? examples/s]

Saving the dataset (0/9 shards):   0%|          | 0/459111 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 1836440
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 459111
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 2279
    })
})

In [9]:
tokenizer.model_max_length

512

In [3]:
chunk_size = 512

In [11]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 551'
'>>> Review 1 length: 17'
'>>> Review 2 length: 304'


In [12]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 872'


In [13]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 512'
'>>> Chunk length: 360'


In [15]:
def group_texts(examples):
    chunk_size = 512
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result


In [16]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True, num_proc=16)
lm_datasets.save_to_disk('distilbert-model/lm_dataset-30000')
lm_datasets

Map (num_proc=16):   0%|          | 0/1836440 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/459111 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/2279 [00:00<?, ? examples/s]

Saving the dataset (0/55 shards):   0%|          | 0/2509639 [00:00<?, ? examples/s]

Saving the dataset (0/14 shards):   0%|          | 0/626095 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3075 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2509639
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 626095
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3075
    })
})

In [65]:
tokenizer.decode(lm_datasets["train"][1234]["input_ids"])

'შემთხვევა [SEP] [CLS] საქართველოს მთავრობისა და სპორტისა და ახალგაზრდობის საქმეთა სამინისტროს სტრატეგიისა და პრიორიტეტული მიმართულებების, ასევე საქართველოს ახალგაზრდული პოლიტიკის გათვალისწინებით, მიმდინარე წელს სსიპ „ ბავშვთა და ახალგაზრდობის განვითარების ფონდის “ პროგრამულ პრიორიტეტებად შემდეგი მიმართულებები განისაზღვრა : ა ) დასაქმების ხელშეწყობა და არაფორმალური განათლება : ა. ა ) სოციალური მეწარმეობის ხელშეწყობა ( შენიშვნა : საქართველოს სამოქალაქო კოდექსის 25 - ე მუხლის მე - 5 პუნქტი – არასამეწარმეო ( არაკომერციული ) იურიდიული პირი უფლებამოსილია ეწეოდეს დამხმარე ხასიათის სამეწარმეო საქმიანობას, რომლიდან მიღებული მოგებაც უნდა მოხმარდეს არასამეწარმეო ( არაკომერციული ) იურიდიული პირის მიზნების რეალიზებას. ასეთი საქმიანობის შედეგად მიღებული მოგების განაწილება არასამეწარმეო ( არაკომერციული ) იურიდიული პირის დამფუძნებლებს, წევრებს, შემომწირველებს, აგრეთვე ხელმძღვანელობისა და წამომადგენლობითი უფლებამოსილების მქონე პირებს შორის დაუშვებელია. პრიორიტეტი სოციალური მეწარმეობა ნიშნავს კანონმდებ

In [8]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

'ხადიჯა ისმაილოვას პირადი ცხოვრების ამსახველი უკანონო ჩანაწერები. თუმცა გამომძიებელი ჟურნალისტი არ აპირებს ფარ - ხმლის დაყრას და აცხადებს, რომ მოვლენათა ნებისმიერი განვითარების შემთხვევაში არ აპირებს საზღვარგარეთ დარჩენას. „ მისი დაპატიმრება იქნება უძლიერესი დარტყმა ხელისუფლების ზეწოლის ქვეშ მყოფ აზერბაიჯანულ მედიასა და სამოქალაქო სექტორზე “, - ამბობს რებეკა ვინსენტი, აშშ - ის ყოფილი დიპლომატი და ამჟამად უფლებადამცველი აქტივისტი. ხადიჯა ისმაილოვა, რომელიც სტრასბურგში იმყოფებოდა მივლინებით, 3 ოქტომბერს ბრუნდება ბაქოში. ამავე თემაზე [SEP] [CLS] ჟანა ტარხანოვა, ვლადიკავკაზი - ცხინვალი [SEP] [CLS] პაატა ბურჭულაძე : “ ვაი, ჩემო საქართველო ” • artinfo. ge ოპერის მომღერალი პაატა ბურჭულაძე ქვეყანაში მიმდინარე მოვლენებს აფასებს : “ როგორც იქნა, გამოირთო “ რუსთავი 2 ” და ხომ ატყობთ რა საამო სიმშვიდეა ქვეყანაში. აღარავინ ტეხს განგაშს, რომ კორუფციაა და სიღარიბე, აღარც ოკუპაციის თემა მუსირებს. აგერ, ია კერძაიას სიცოცხლის მოსპობაში ბრალეული მხარე ვერ იქნა აღმოჩენილი და საქმე დახურეს. სამაგიეროდ, მამ

In [9]:
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
wholde_word_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm_probability=0.15)

In [13]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in wholde_word_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] ხადიჯა ისმაილოვა : ხუც მათ დღის სინათლეზე გამოვიყვან მედია [MASK] ისმაილოვა : მე მათ დღის სინათლეზე გამოვიყვან ოქტომბერი 03, [MASK] აზერბაიჯანელი გამომძიებელი ჟურნალისტი ხადიჯა [MASK] ამბობს, რომ მას, სამშობლოში [MASK] შემთხვევაში, დაპატიმრება ემუქრება. ისმაილოვას [MASK] განცხადების ფონი [MASK] ზნე [MASK] მდიდარ აზერბაიჯანში დამოუკიდებელი ჟურნალისტებისა და უფლებადამცველი [MASK] [MASK] დევნა, რასაც საერთაშორისო საზოგადოების [MASK] არაერთი სპროტესტო განცხადება [MASK] [MASK] ხადიჯა ისმაილოვა, რომელიც [MASK] თავისუფლების აზერბაიჯანულ სამსახურში დილის პროგრამას უძღვება [MASK] პრეზიდენტ ილ [MASK] [MASK] [MASK] [MASK] და [MASK] ოჯახისmav ბიზნესინტერესების შესახებ [MASK] [MASK] მწვავე პუბლიკაციით არის ცნობილი. როგორც ხადიჯა ამბობს, მის წინააღმდეგ vax მუხლით აღძრულია სისხლის სამართლის საქმე და 3 ოქტომბერს სასამართლოშია გამოძახებული : „ გაფრთხილებული ვარ, [MASK] მივლინებიდან დაბრუნების შემდეგ დამაპატიმრებენ, რაც დაშინების [MASK] მეთოდია. დარწმუნებული ვარ, მათ სურთ ან არ დავბრუნდე აზე

In [26]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")
        # სიტყვებსა და ტოკენებს შორის კავშირების დამყარება
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # სიტყვების შემთხვევითად შენიღბვა
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [22]:
samples = [lm_datasets["train"][i] for i in range(3)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

features: [{'input_ids': [2, 16885, 19765, 30, 431, 562, 1419, 13774, 403, 5013, 1746, 6014, 16885, 19765, 30, 431, 562, 1419, 13774, 403, 5013, 1746, 3281, 1702, 16, 1598, 28060, 240, 1401, 23226, 7840, 16885, 19765, 2222, 16, 400, 595, 16, 8891, 8281, 1095, 16, 11950, 776, 9942, 18, 19765, 250, 421, 8496, 25388, 243, 7321, 692, 6819, 364, 14458, 5395, 16472, 243, 367, 24859, 240, 10563, 2074, 11640, 16, 2559, 1323, 3658, 2727, 6171, 840, 16914, 245, 1571, 7759, 18, 16885, 19765, 16, 620, 6491, 3627, 8840, 391, 7063, 15793, 15204, 18492, 382, 16, 2386, 5878, 10409, 3911, 1503, 243, 367, 666, 3491, 9426, 9090, 242, 652, 381, 386, 855, 9168, 3858, 5507, 11775, 2804, 543, 2090, 18, 659, 16885, 2222, 16, 661, 1534, 28698, 7627, 28434, 243, 2418, 3330, 1183, 367, 23, 9117, 9125, 243, 14999, 841, 393, 30, 185, 10616, 446, 1172, 16, 400, 22764, 1330, 633, 8281, 638, 2653, 7669, 14005, 16, 650, 29888, 386, 10904, 7055, 243, 18, 5728, 1172, 16, 562, 10811, 478, 389, 10131, 807, 14458, 16, 1007

In [29]:
from transformers import DataCollatorForWholeWordMask

samples = [lm_datasets["train"][i] for i in range(2)]
batch = DataCollatorForWholeWordMask(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

AttributeError: 'list' object has no attribute 'mask_token'

In [39]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [14]:
lm_datasets["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 2509639
})

In [25]:
from datasets import concatenate_datasets

tf_train_dataset = model.prepare_tf_dataset(
    lm_datasets["train"],
    collate_fn=wholde_word_collator,
    shuffle=True,
    batch_size=32,
)

eval_dataset = concatenate_datasets([lm_datasets["test"], lm_datasets["validation"]])
tf_eval_dataset = model.prepare_tf_dataset(
    eval_dataset,
    collate_fn=wholde_word_collator,
    shuffle=False,
    batch_size=32,
)

In [18]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [26]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-mc4-cleaned-52000", tokenizer=tokenizer
)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
C:\Users\Davit6174\PycharmProjects\pythonProject\distilbert-base-uncased-finetuned-mc4-cleaned-52000 is already a clone of https://huggingface.co/Davit6174/distilbert-base-uncased-finetuned-mc4-cleaned-52000. Make sure you pull the latest changes with `repo.git_pull()`.


In [21]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

  25/4916 [..............................] - ETA: 38:31 - loss: 14.0956

KeyboardInterrupt: 

In [27]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset)

78426/78426 [==============================] - 29664s 378ms/step - loss: 4.5141 - val_loss: 3.9369


In [28]:
model.save_pretrained('distilbert-base-uncased-fine-tuned-cleaned')

In [30]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model=model, tokenizer=tokenizer
)

In [62]:
# text = 'ძალიან [MASK] საჭმელია, სულ მინდება'
text = 'გამარჯობა, მე დავითი [MASK]'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}", round(pred['score'], 3))

>>> გამარჯობა, მე დავითი ვარ 0.279
>>> გამარჯობა, მე დავითი : 0.108
>>> გამარჯობა, მე დავითი - 0.088
>>> გამარჯობა, მე დავითი, 0.065
>>> გამარჯობა, მე დავითი. 0.059
